# Abell 1656: the Coma Cluster of Galaxies
This notebook is based on the EURO-VO tutorial: "Abell 1656: The Coma Cluster of Galaxies" (http://www.euro-vo.org/?q=science/scientific-tutorials) and shows how to perform steps 1 to 8 from within a Jupyter notebook. Steps 9 to 11 are not yet implemented. 

## Introduction
The goals of this notebook tutorial are:
 - Examine the Coma cluster of galaxies (Abell 1656) using VO data and tools withinin a jupyter notebook in order to perform a quick evaluation of the mean redshift and velocity dispersion of the cluster.
 - Use redshifts and photometry (Petrosian r magnitude) of the SDSS survey and then add redshifts of the CAIRNS survey (Rines et al. 2003) in order to improve the completeness of the redshift sample.
 
For our analysis we need to following python packages:

In [2]:
import ipyaladin.aladin_widget as ipyal
from astroquery.vizier import Vizier
from astroquery.xmatch import XMatch
from astropy.table import Table, vstack
from astropy import units as u
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np

In [3]:
%ipyal inline

UsageError: Line magic function `%ipyal` not found.


## Display the region of Abell 1656 in Aladin lite
We start by displaying the Coma Cluster in an Aladin Lite widget. We will centre the DSS2 colour images (`survey='P/DSS2/color'`) on Abell 1656 (`target='A1656'`) and set the field of view to 0.7deg (`fov=0.7`). At the distance of Coma, our field of view corresponds to approximately 1Mpc a region large enough for our purposes. 

Note that if you are using Jupyter lab you can now open a second python3 notebook, click on the "Python 3" button in the top, right corner of this notebook to switch kernel. A new window will pop up and you can then select this notebook (i.e. "Abel1656...") as kernel. This will link the two notebooks such that they see the same variables ect. You may use the second linked notebook to be able to look at a Aladin Lite widget the entire time, while doing the analysis work in this notebook. Please see the ipyaladin webpage for more information on how to use the widget in Jupyter lab. 

In [4]:
aladin = ipyal.Aladin(target='A1656', fov=0.7, survey='P/DSS2/color')

In [5]:
aladin

Aladin(fov=0.7, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'ov…

As with any Aladin Lite implementation, you can interact with this widget: 
 - to zoom in and out place you mouse pointer on top of the image and scroll. 
 - with <img src="Images/ipyaladin_layer.png" alt="the Layer Button" style="width:30px; display: inline-block;"/>  you can select other image surveys and manage the current view.
 - if you like to look at another target, you can use the search field <img src="Images/ipyaladin_search.png" alt="the Search Button" style="width:30px; display: inline-block;"/> to get there. 

These interactions can also be steered by changing properties of the variable `aladin`. If for example you wanted to focus on M101 instead of the Antennae galaxies, do:

## Load the SDSS-DR9 catalog and select galaxies

In [ ]:
catalog_list = Vizier.find_catalogs('SDSS DR9')
print({k:v.description for k,v in catalog_list.items()})

Now, we query "The SDSS Photometric Catalog, Release 9 (Adelman-McCarthy+, 2012)". To do so we increase the limit of loaded rows from 50 (default set-up) to as many as there are. 

In [ ]:
Vizier.ROW_LIMIT = -1
results1 = Vizier.query_region("A1656", radius="0d40m0s", catalog='V/139')
print(results1)

As you can see the result of our query includes data from one catalogue for 37389 objects (i.e. the catalogue has 37389 rows). The output from this query is a list of astropy.table.table.Table objects. We now select the 0th table in this list, restrict the table to objects, which are galaxies ('cl' = 3) and also SDSS primary sources ('mode' = 1), and use only the columns named 'RA_ICRS', 'DE_ICRS', 'rmag' and 'zsp'. 

In [ ]:
sdss_dr9 = results1[0][(results1[0]['cl']==3) & (results1[0]['mode']==1)]
sdss_dr9 = sdss_dr9['RA_ICRS', 'DE_ICRS', 'rmag', 'zsp']
sdss_dr9

## Identify the brightest sources as being stars contaminating the sample
Next we would like to exclude stars from our sample. To do so we select the brightest sources ('rmag' < 11.7) and check with the Aladin lite widget that we actually only exclude stars. 

In [ ]:
stars = sdss_dr9[sdss_dr9['rmag'] < 11.5]
aladin.add_table(stars)
print('Our sample contains {} stars.'.format(len(stars)))

In the Aladin lite widget you will need to zoom out to properly see all the brightest sources, but looking at each source, will show you that indeed these are all stars. 

## Build a subset of galaxies with photometry and redshift in SDSS
Now on to exploring the galaxies in our sample. 
We again build a subset, this time for all sources fainter than 11.5mag (to leave out the stars identified in the section above) but brighter than 17.77mag, which is the completeness limit the SDSS spectroscopic sample. 

In [ ]:
zsp17 = sdss_dr9[(sdss_dr9['rmag'] > 11.5) & (sdss_dr9['rmag'] < 17.77)]
zsp17

## Improve the completeness with other sources of redshifts in Vizier
As you can see, not all galaxies in our zsp17 sample have redshift measurements (some rows have '--' in the 'zsp' column, i.e. they are masked). So in order to improve the completeness of our sample we will now use Vizier to search for redshifts in the Rines et al (2003) catalogue. 

In [ ]:
catalog_list = Vizier.find_catalogs('redshifts Rines 2003')
print({k:v.description for k,v in catalog_list.items()})

Of these two catalogues, we are interested in the 'J/AJ/126/2152' catalogue, more specifically in their 'galaxies' subtable. So after dividing the zsp17 into two subsamples (the ones with zsp measurements - zsp17_with - and the ones without zsp measurements - zsp17_wihtout), we will use the `astroquery.XMatch.query` module to look for crossmatches of galaxies from the zsp17_wihtout sample in the 'J/AJ/126/2152/galaxies' hosted at Vizier.

In [ ]:
ind = (zsp17['zsp'] < 5.) & (zsp17['zsp'] > 0.) # measured redshifts are between 0. and 2.
zsp17_with = zsp17[ind]
zsp17_without = zsp17[~ind]
zsp17_without.write('zsp17_without.csv')
table = XMatch.query(cat1=open('zsp17_without.csv'), 
                     cat2='vizier:J/AJ/126/2152/galaxies',
                     max_distance=5 * u.arcsec, colRA1='RA_ICRS',
                     colDec1='DE_ICRS')
table

## Build the final catalogue including the Rines et al. (2003) redshifts
With this we now have cz measurements for 27 additional galaxies. Now let's add these data to the zsp17_with table.

In [ ]:
zsp17_with['cz'] = zsp17_with['zsp'] * 300000
zsp17_without = table[zsp17_with.colnames]
zsp17_final = vstack([zsp17_with, zsp17_without])
zsp17_final

## Determine the cz distribution, the cluster average velocity and velocity dispersion

In [ ]:
sns.set_style("darkgrid")
fig = plt.figure(figsize=(7.5, 6.0))
ax = fig.add_axes([0.17, 0.17, 0.77, 0.77])
sns.distplot(zsp17_final['cz'], rug=True, ax=ax, kde=False)

Note how there is a large range of recession velocities in our sample. We are interested in the range of recession velocities of the Coma cluster. To focus on these, we restrict ourselfs to recession velocities between 3000 and 11000 km/s

In [ ]:
sns.set_style("darkgrid")
fig = plt.figure(figsize=(7.5, 6.0))
ax = fig.add_axes([0.17, 0.17, 0.77, 0.77])
ind = (zsp17_final['cz'] > 3000.) & (zsp17_final['cz'] < 11000.)
zsp17_coma = zsp17_final[ind]
sns.distplot(zsp17_coma['cz'], rug=True, ax=ax, kde=False)
ax.set_xlim([3000, 11000])


Now let's get some statistics:

In [ ]:
print('The mean velocotity in Coma is: {0:.1f}km/s and \
the standard deviation (i.e. velocity dispersion):\
{1:.1f}km/s'.format(np.mean(zsp17_coma['cz']), np.std(zsp17_coma['cz'])))

This is in good agreement with more refined analyses. 

## Summary
In the original tutorial there are three more sections left:
    9. Look for HST spectra in the Coma Cluster
    10. Visualise and analyse the HST spectrum with CASSIS
    11. Fit a gaussian and a continuum to the hydrogen line
Since the server selector in Aladin is different now, I was not able to find the spectra and do the remaining tutorial. Thus, I am also not yet able to convert this part of the tutorial to this jupyter notebook. 

For sections 1. to 8. however, it was possible to translate the tutorial to this jupyter notebook using the ipyaladin widget and the astroquery and astropy packages together with standard python plotting packages (matplotlib and seaborn).